In [1]:
import glob
import os
import timm 
import torch

# I'm adding this in
from torch.autograd import Variable

import numpy as np
import pandas as pd
from PIL import Image
import efficientnet.keras as efn

from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Subset
from torchvision import transforms, models, datasets
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from sklearn import preprocessing


METADATA_SUBSET_PATH = "/Users/franceskoback/Documents/xray_test/metadata_100subset_df.csv"

def get_manufacturer_labels(encoder, target_variable = "(0008, 0070) Manufacturer"):
    df = pd.read_csv(METADATA_SUBSET_PATH)
    df["id"] = df["id"].astype("str").str.zfill(8)
    df["code"] = encoder.fit_transform(df[target_variable])
    
    return {row["id"]: row["code"] for i, row in df.iterrows()}

class CustomImageDataset(Dataset):
    def __init__(self):
        #self.img_dir = "/Users/franceskoback/Documents/research/pytorch_1/imagezz"
        self.img_dir = "/Users/franceskoback/Documents/xray_test/xray_subsets"

        self.images = glob.glob(os.path.join(self.img_dir, "*.npy")) 
        self.le = preprocessing.LabelEncoder()
        self.label_map = get_manufacturer_labels(self.le)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.fromarray(np.load(img_path)).convert("RGB")
        image = transforms.ToTensor()(image)
        xray_id = os.path.basename(img_path).replace(".npy", "")
        
        return {"image": image, "label": self.label_map[xray_id]}

def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [2]:
dataset = CustomImageDataset()
datasets = train_val_dataset(dataset)
print(len(datasets['train'].dataset)) #6
#datasets['train'].dataset, batch_size=params["batch_size"], shuffle=True
train_loader = DataLoader(
    datasets['train'].dataset, batch_size=3, shuffle=True
)
valid_loader = DataLoader(
    datasets['val'].dataset, batch_size=3, shuffle=True
)
print(len(train_loader.dataset)) #6 
len_train=len(datasets['train'])
len_val= len(datasets['val'])
print("Training length", len(datasets['train']))
print("Validation length", len(datasets['val']))

100
100
Training length 75
Validation length 25


In [3]:
def Net(num_classes):
    model = models.resnet50(pretrained=True)
    #model=timm.create_model(model,pretrained= True)
    #model = EfficientNet.from_pretrained('efficientnet-b0')
    #model= efn.EfficientNetB0(weights='imagenet')
    # Freeze parameters so we don't backprop through them
    for param in model.parameters():
        param.requires_grad = False

    from collections import OrderedDict
    classifier = nn.Sequential(OrderedDict([
                              ('fc1', nn.Linear(2048, 1024)),
                              ('relu', nn.ReLU()),
                              ('fc2', nn.Linear(1024, 256)),
                              ('relu', nn.ReLU()),
                              ('fc3', nn.Linear(256, num_classes)),
                              ('output', nn.LogSoftmax(dim=1))
                              ]))

    model.fc = classifier
    return model

params = {
    "model": "resnet50",
    #"device": "cuda",
    "lr": 0.001,
    "batch_size": 3, #64
    "num_workers": 1, #20
    "n_epochs": 50, #100
    "image_size": 224, 
    "in_channels": 3, #3
    "num_classes": 3, #12
    "device": "cpu"
}

model = Net(params['num_classes'])
model.to(params["device"])
#loss_fn = nn.NLLLoss() # we want MSE loss i think 
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = params['lr'])

/Users/franceskoback/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/franceskoback/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
def train_one_epoch(epoch, model, loss_fn, optimizer, train_loader, device = "cpu"):
    #put model in training state
    model.train()
    #i=0
    train_loss = 0.0
    

    for batch_idx, img_dicts in enumerate(train_loader,0):  #used to be enumerate(train_loader)
        inputs = img_dicts["image"] #ORIGINAL ONE
        labels = img_dicts["label"]  # ORIGINAL ONE
        
        inputs = Variable(inputs.to(device).float())
        labels = Variable(labels.to(device).float())
        #print(inputs.shape, labels.shape)
        
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad() # do we need this or not? sets all grads to None 

        # print statistics
        #running_loss += loss.item()
        #if i % 2000 == 1999:    # print every 2000 mini-batches
         #   print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            #running_loss=0
          #  i=i+1

        
        
        #
    
        train_loss+= ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
        if batch_idx%5==0:
            print('train loss', train_loss)
    
    #print('Epoch {} avg Training loss: {:.3f}'.format(epoch+1, train_loss))
    
    return model, train_loss

def test_one_epoch(epoch, model, loss_fn, loader, len_val, device = "cpu"):
    model.eval()
    
    #pbar = tqdm(enumerate(test_loader), total = len(test_loader))
    running_loss = 0
    actual_labels = []
    pred_labels = []
    
    #for step, (imgs, labels) in pbar:
    for batch_idx, img_dicts in enumerate(loader,0):    
        inputs = img_dicts["image"] #ORIGINAL ONE
        labels = img_dicts["label"]  # ORIGINAL ONE
        
        inputs = Variable(inputs.to(device).float())
        labels = Variable(labels.to(device).float())
        
        log_preds = model(inputs)
        loss = loss_fn(log_preds, labels)
        
        preds = torch.exp(log_preds)
        running_loss+=((1 / (batch_idx + 1)) * (loss.data.item() - running_loss))
        
        #calculate accuracy
        top_prob, top_class = preds.topk(1, dim=1)
        pred_labels+= list((top_class.view(-1)).cpu().numpy())
        actual_labels+= list(labels.cpu().numpy())
        
        
    
    accuracy = ((np.array(pred_labels)==np.array(actual_labels)).sum())/len_val #size of test set
    correct = ((np.array(pred_labels)==np.array(actual_labels)).sum())
    total = len_val
    
    
    return running_loss, accuracy, correct, total

## Training Loop

In [5]:
train_losses = []
valid_losses = []

for epoch in range(params['n_epochs']):
    train_loss = train_one_epoch(epoch, model, loss_fn, optimizer, train_loader)
    train_losses+= [train_loss]
    valid_loss, accuracy, correct, total = test_one_epoch(epoch, model, loss_fn, valid_loader, len_val)
    valid_losses+=[valid_loss]
    print('Epoch {} avg Valid loss: {:.3f}'.format(epoch+1, valid_loss))
    print('Epoch {} Valid accuracy: {:.1%} ({} of {} right)\n'.format(epoch+1, accuracy, correct, total))
    if len(valid_losses)>1 and (valid_loss<min(valid_losses[:-1])):
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss_fn,
            }, 'checkpoint.tar')

    


/Users/franceskoback/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


train loss 45.04386901855469
train loss 35.91239229838053
train loss 25.223380608992144
train loss 25.93820208311081
train loss 26.949111121041433
train loss 30.232530515927536
train loss 34.17260255736689


/Users/franceskoback/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1 avg Valid loss: 31.362
Epoch 1 Valid accuracy: 68.0% (17 of 25 right)

train loss 36.46467590332031
train loss 34.94554154078166
train loss 33.49651033228094
train loss 33.08366608619691
train loss 31.852205208369668
train loss 29.74642079151594
train loss 32.368195922144
Epoch 2 avg Valid loss: 32.585
Epoch 2 Valid accuracy: 76.0% (19 of 25 right)

train loss 3.2914562225341797
train loss 26.406837463378906
train loss 23.64210785518993
train loss 21.4831165522337
train loss 28.852644182386857
train loss 30.587758972094615
train loss 31.615715788256743
Epoch 3 avg Valid loss: 33.084
Epoch 3 Valid accuracy: 76.0% (19 of 25 right)

train loss 3.262333631515503
train loss 34.5616055727005
train loss 33.910685690966524
train loss 31.347480580210686
train loss 30.23035387765794
train loss 31.999841121526867
train loss 31.74843469742806
Epoch 4 avg Valid loss: 31.396
Epoch 4 Valid accuracy: 28.0% (7 of 25 right)

train loss 52.93706130981445
train loss 41.68382581075033
train loss 35

In [6]:
# load the model that got the best validation accuracy
checkpoint = torch.load('checkpoint.tar')
loaded_model = Net(params['num_classes'])
loaded_model.to(params["device"])
loaded_model.load_state_dict(checkpoint['model_state_dict'])

loaded_criterion = checkpoint['loss']

#optimizer = optim.Adam(model.parameters(), lr = 0.003)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

last_epoch = checkpoint['epoch']+1

test_loss, accuracy, correct, total = test_one_epoch(None, loaded_model, loaded_criterion, valid_loader, len_val)

print('Test loss: {:.3f}'.format(test_loss))
print('Test accuracy: {:.1%} ({} of {} right)\n'.format(accuracy, correct, total))
    

Test loss: 31.517
Test accuracy: 76.0% (19 of 25 right)

